In [1]:
import requests
import json
import re
import os
import time
from src.navegador.navegador import Navegador
from selenium.webdriver.common.by import By
from src.utils import utils
from pathlib import Path

In [ ]:
class Pipefy(object):
    """ Integration class with Pipefy rest api. """

    def __init__(self, token=None, login=None, senha=None):
        if token:
            self.token = token if 'Bearer' in token else 'Bearer %s' % token
        else:
            self.token = None

        self.login = login
        self.senha = senha
        
        self.headers = {'Content-Type': 'application/json', 'Authorization': self.token}
        self.endpoint = 'https://app.pipefy.com/graphql'

    def fazer_login(self,  link=None, caminho_downloads=None, options=None):
        #Verificar se Caminho_Downloads foi preenchido e caso não, utilizar download Padrão do Windows
        if not caminho_downloads:
            caminho_downloads = str(Path.home() / "Downloads")

        #Verificar se Link foi preenchido e caso não, utilizar Link Padrão do Pipefy
        if not link:
            link = 'https://app.pipefy.com'

        #Abrir Navegador utilizando Link definido
        navegador = Navegador(link, caminho_downloads=caminho_downloads, options=options)

        #Preencher Login
        navegador.identifica_elemento_name('username').send_keys(self.login)

        #Clica no botão continuar
        navegador.identifica_elemento_id('kc-login').click()

        #Preencher Senha
        navegador.identifica_elemento_name('password').send_keys(self.senha)

        #Clicar no Botão Enviar
        navegador.identifica_elemento_id('kc-login').click()

        #Troca para a aba pricipal do código
        navegador.switch_to.window(navegador.window_handles[0]) 

        #Retorna o Object Navegador 
        return navegador
    
    def baixar_relatorio_pipefy_rpa(self, id_report, id_pipe, caminho_download=None, options=None):

        #Verificar Caminho_Downloads
        if not caminho_download:
            caminho_download = utils.obter_caminho_downloads()

        #Define quantidade de Arquivos Antes do Download
        Quantidade_Arquivos_Antes = len(os.listdir(caminho_download))

        #Definir Link do relatório
        link_report = f"https://app.pipefy.com/pipes/{str(id_pipe)}/reports_v2/{str(id_report)}"

        #Abrir Navegador utilizando Link do Relatório
        navegador = self.fazer_login(link=link_report, caminho_downloads=caminho_download, options=options)

        #Clicar no botão Exportar
        navegador.clica_elemento_xpath("//button[@data-testid='report-export']")

        #Clicar no botão Download
        navegador.clica_elemento_xpath("//button[@title='Download']")

        #Aguardar Barra de Carregamento Sumir
        texto_aguardando = 'x'
        while texto_aguardando != "":
            try:
                texto_aguardando = ""
                texto_aguardando = navegador.find_element(By.XPATH, '//div[@class="pp-modals-wrapper "]').text
            except:
                pass
        
        utils.aguardar_arquivo_baixar(Quantidade_Arquivos_Antes=Quantidade_Arquivos_Antes, timeout=360, caminho_downloads=caminho_download)

        #Retorna o último Arquivo Baixado na pasta de Download
        return utils.obter_ultimo_arquivo(caminho_download)

In [3]:
pipefy = Pipefy(login="system.robotic@rjzcyrela.com.br", senha="@Iops.2023")

In [ ]:
arquivo = pipefy.baixar_relatorio_pipefy_rpa(id_pipe=303525543,id_report=300523529)

'C:\\Users\\Lucas.Graca\\Downloads\\relatrio___io_ps_11-04-2025 (1).xlsx'